In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
from datasets import concatenate_datasets
from datasets import DatasetDict

def contains_price(example):
    return ('price' in example['text'].lower() and 'high' in example['text'].lower()) or ('price' in example['text'].lower() and 'low' in example['text'].lower())

def contains_pricehighnegative(example):
    return 'price' in example['text'].lower() and 'high' in example['text'].lower() and example['label'] == 0

def contains_pricehighpositive(example):
    return 'price' in example['text'].lower() and 'high' in example['text'].lower() and example['label'] == 1

def contains_pricelownegative(example):
    return 'price' in example['text'].lower() and 'low' in example['text'].lower() and example['label'] == 0

def contains_pricelowpositive(example):
    return 'price' in example['text'].lower() and 'low' in example['text'].lower() and example['label'] == 1

filtered_dataset = dataset.filter(contains_price)

def change_label(example):
    # Change the label based on your condition
    if example['label'] > 2:
        example['label'] = 1
    else:
        example['label'] = 0
    return example

filtered_dataset = filtered_dataset.map(change_label)

pricehighnegative_train_unfilter = filtered_dataset.filter(contains_price)['train'].shuffle(seed=42).select([i for i in list(range(27000))])

pricehighnegative_train_balance = filtered_dataset.filter(contains_pricehighnegative)['train'].shuffle(seed=42).select([i for i in list(range(2900))])
pricehighpositive_train_balance = filtered_dataset.filter(contains_pricehighpositive)['train'].shuffle(seed=42).select([i for i in list(range(2900))])
pricelownegative_train_balance = filtered_dataset.filter(contains_pricelownegative)['train'].shuffle(seed=42).select([i for i in list(range(2900))])
pricelowpositive_train_balance = filtered_dataset.filter(contains_pricelowpositive)['train'].shuffle(seed=42).select([i for i in list(range(2900))])

pricehighnegative_validation_balance = filtered_dataset.filter(contains_pricehighnegative)['train'].shuffle(seed=42).select([i for i in list(range(2900,3100))])
pricehighpositive_validation_balance = filtered_dataset.filter(contains_pricehighpositive)['train'].shuffle(seed=42).select([i for i in list(range(2900,3100))])
pricelownegative_validation_balance = filtered_dataset.filter(contains_pricelownegative)['train'].shuffle(seed=42).select([i for i in list(range(2900,3100))])
pricelowpositive_validation_balance = filtered_dataset.filter(contains_pricelowpositive)['train'].shuffle(seed=42).select([i for i in list(range(2900,3100))])

pricehighnegative_train_imbalance = filtered_dataset.filter(contains_pricehighnegative)['train'].shuffle(seed=42).select([i for i in list(range(2900))])
pricehighpositive_train_imbalance = filtered_dataset.filter(contains_pricehighpositive)['train'].shuffle(seed=42).select([i for i in list(range(500))])
pricelownegative_train_imbalance = filtered_dataset.filter(contains_pricelownegative)['train'].shuffle(seed=42).select([i for i in list(range(500))])
pricelowpositive_train_imbalance = filtered_dataset.filter(contains_pricelowpositive)['train'].shuffle(seed=42).select([i for i in list(range(2900))])

newdataset_train_unfilter = pricehighnegative_train_unfilter
newdataset_train_balance = concatenate_datasets([pricehighnegative_train_balance, pricehighpositive_train_balance, pricelownegative_train_balance, pricelowpositive_train_balance])
newdataset_train_imbalance = concatenate_datasets([pricehighnegative_train_imbalance, pricehighpositive_train_imbalance, pricelownegative_train_imbalance, pricelowpositive_train_imbalance])
newdataset_validation = concatenate_datasets([pricehighnegative_validation_balance, pricehighpositive_validation_balance, pricelownegative_validation_balance, pricelowpositive_validation_balance])

'''
Test tests for four groups
'''
pricehighnegative_test = filtered_dataset.filter(contains_pricehighnegative)['test'].shuffle(seed=42).select([i for i in list(range(230))]) #group 1
pricehighpositive_test = filtered_dataset.filter(contains_pricehighpositive)['test'].shuffle(seed=42).select([i for i in list(range(230))]) #group 2
pricelownegative_test = filtered_dataset.filter(contains_pricelownegative)['test'].shuffle(seed=42).select([i for i in list(range(230))]) #group 3
pricelowpositive_test = filtered_dataset.filter(contains_pricelowpositive)['test'].shuffle(seed=42).select([i for i in list(range(230))]) #group 4

'''
Final Dataset
'''
final_dataset_unfilter = DatasetDict({
    'train': newdataset_train_unfilter,
    'validation': newdataset_validation,
    'test_group_1': pricehighnegative_test,
    'test_group_2': pricehighpositive_test,
    'test_group_3': pricelownegative_test,
    'test_group_4': pricelowpositive_test,
})

final_dataset_balance = DatasetDict({
    'train': newdataset_train_balance,
    'validation': newdataset_validation,
    'test_group_1': pricehighnegative_test,
    'test_group_2': pricehighpositive_test,
    'test_group_3': pricelownegative_test,
    'test_group_4': pricelowpositive_test,
})

final_dataset_imbalance = DatasetDict({
    'train': newdataset_train_imbalance,
    'validation': newdataset_validation,
    'test_group_1': pricehighnegative_test,
    'test_group_2': pricehighpositive_test,
    'test_group_3': pricelownegative_test,
    'test_group_4': pricelowpositive_test,
})
print(final_dataset_unfilter)
print(final_dataset_balance)
print(final_dataset_imbalance)


Filter:   0%|          | 0/650000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/40779 [00:00<?, ? examples/s]

Map:   0%|          | 0/3089 [00:00<?, ? examples/s]

Filter:   0%|          | 0/40779 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/40779 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3089 [00:00<?, ? examples/s]

Filter:   0%|          | 0/40779 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3089 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 27000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 800
    })
    test_group_1: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
    test_group_2: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
    test_group_3: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
    test_group_4: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
})
DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 11600
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 800
    })
    test_group_1: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
    test_group_2: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
    test_group_3: Dataset({
        feature

#Pretrained Albert

In [5]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00


In [8]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification, pipeline
import torch

# Load the pre-trained ALBERT configuration, tokenizer, and model
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2')

# Initialize the sentiment analysis pipeline
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, truncation=True, padding=True)

# Analyzing sentiments in the dataset
results = []
for i in range(1, 5):
    results.append(classifier(final_dataset_unfilter['test_group_'+str(i)]["text"]))


spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for i in range(len(results)):
  predicted_labels = [int(pred['label'].split('_')[-1]) for pred in results[i]]

  true_labels = final_dataset_unfilter['test_group_'+str(i+1)]['label']  # Replace 'labels' with the actual label column name
  accuracy = sum([pred == true for pred, true in zip(predicted_labels, true_labels)]) / len(true_labels)
  print(f"Group {i+1} Accuracy: {accuracy:.4f}")


Group 1 Accuracy: 0.0348
Group 2 Accuracy: 0.9870
Group 3 Accuracy: 0.0261
Group 4 Accuracy: 0.9565


#train

In [ ]:
!pip uninstall accelerate transformers

Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-estimate-memory
    /usr/local/bin/accelerate-launch
    /usr/local/lib/python3.10/dist-packages/accelerate-0.25.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/accelerate/*
Proceed (Y/n)? Y
  Successfully uninstalled accelerate-0.25.0
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.35.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.35.2


In [ ]:
!pip install accelerate
!pip install evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


##Unfilter

In [5]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification, AlbertConfig, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import numpy as np
import evaluate

# ALBERT Configuration
configuration = AlbertConfig.from_pretrained('albert-base-v2')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', config=configuration)

# Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training Arguments
training_args = TrainingArguments(
    "test-trainer",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=False,  # Set this to False
    save_strategy="no",  # No intermediate saving
)

# Tokenization Function
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

# Apply Tokenization
tokenized_datasets_unfilter = final_dataset_unfilter.map(tokenize_function, batched=True)

# Trainer Configuration
trainer_unfilter = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets_unfilter["train"],
    eval_dataset=tokenized_datasets_unfilter["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Training
trainer_unfilter.train()

# Prediction
predictions_test_group_1 = trainer_unfilter.predict(tokenized_datasets_unfilter["test_group_1"])
predictions_test_group_2 = trainer_unfilter.predict(tokenized_datasets_unfilter["test_group_2"])
predictions_test_group_3 = trainer_unfilter.predict(tokenized_datasets_unfilter["test_group_3"])
predictions_test_group_4 = trainer_unfilter.predict(tokenized_datasets_unfilter["test_group_4"])

# Processing Predictions
preds_test_group_1 = np.argmax(predictions_test_group_1.predictions, axis=-1)
preds_test_group_2 = np.argmax(predictions_test_group_2.predictions, axis=-1)
preds_test_group_3 = np.argmax(predictions_test_group_3.predictions, axis=-1)
preds_test_group_4 = np.argmax(predictions_test_group_4.predictions, axis=-1)

# Evaluation
metric = evaluate.load("glue", "mrpc")
print("group 4")
results = metric.compute(predictions=preds_test_group_4, references=predictions_test_group_4.label_ids)
print(results)


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.428900,0.362586
2,0.355900,0.272315
3,0.153300,0.249195


group 4
{'accuracy': 0.782608695652174, 'f1': 0.878048780487805}


In [9]:
predictions_test_groups = []
for i in range(1, 5):
    predictions = trainer_unfilter.predict(tokenized_datasets_unfilter[f"test_group_{i}"])
    predictions_test_groups.append(predictions)

metric = evaluate.load("glue", "mrpc")
for i, predictions in enumerate(predictions_test_groups, start=1):
    preds = np.argmax(predictions.predictions, axis=-1)
    print(f"group {i}")
    print(metric.compute(predictions=preds, references=predictions.label_ids))

group 1
{'accuracy': 0.9347826086956522, 'f1': 0.0}
group 2
{'accuracy': 0.8130434782608695, 'f1': 0.8968824940047961}
group 3
{'accuracy': 0.9043478260869565, 'f1': 0.0}
group 4
{'accuracy': 0.782608695652174, 'f1': 0.878048780487805}


##Balance

In [ ]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import evaluate
import numpy as np

# Load the pre-trained ALBERT configuration, tokenizer, and model
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments("test-trainer")

# Tokenization function for dataset
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

# Assuming final_dataset_balance is already defined and formatted properly
tokenized_datasets_balance = final_dataset_balance.map(tokenize_function, batched=True)

trainer_balance = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets_balance["train"],
    eval_dataset=tokenized_datasets_balance["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer_balance.train()

# Predictions for each test group
predictions_test_groups = []
for i in range(1, 5):
    predictions = trainer_balance.predict(tokenized_datasets_balance[f"test_group_{i}"])
    predictions_test_groups.append(predictions)

# Evaluation using the metric
metric = evaluate.load("glue", "mrpc")
for i, predictions in enumerate(predictions_test_groups, start=1):
    preds = np.argmax(predictions.predictions, axis=-1)
    print(f"group {i}")
    print(metric.compute(predictions=preds, references=predictions.label_ids))


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/11600 [00:00<?, ? examples/s]

Step,Training Loss
500,0.604200
1000,0.636400
1500,0.601000
2000,0.500600
2500,0.500900
3000,0.447600
3500,0.434100
4000,0.428000


Checkpoint destination directory test-trainer/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-3500 already exists and is non-empty.Saving will proceed

group 1
{'accuracy': 0.8565217391304348, 'f1': 0.0}
group 2
{'accuracy': 0.8260869565217391, 'f1': 0.9047619047619047}
group 3
{'accuracy': 0.8608695652173913, 'f1': 0.0}
group 4
{'accuracy': 0.8043478260869565, 'f1': 0.891566265060241}


In [ ]:
print("group 2")
metric.compute(predictions=preds_test_group_2, references=predictions_test_group_2.label_ids)

group 2


{'accuracy': 0.0, 'f1': 0.0}

In [ ]:
print("group 3")
metric.compute(predictions=preds_test_group_3, references=predictions_test_group_3.label_ids)

group 3


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


{'accuracy': 1.0, 'f1': 0.0}

In [ ]:
print("group 4")
metric.compute(predictions=preds_test_group_4, references=predictions_test_group_4.label_ids)

group 4


{'accuracy': 0.0, 'f1': 0.0}

#Imbalance

In [ ]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import evaluate
import numpy as np

# Load the pre-trained ALBERT tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments("test-trainer")

# Tokenization function for dataset
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

# Assuming final_dataset_imbalance is already defined and formatted properly
tokenized_datasets_imbalance = final_dataset_imbalance.map(tokenize_function, batched=True)

trainer_imbalance = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets_imbalance["train"],
    eval_dataset=tokenized_datasets_imbalance["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer_imbalance.train()

# Predictions for each test group
predictions_test_groups = []
for i in range(1, 5):
    predictions = trainer_imbalance.predict(tokenized_datasets_imbalance[f"test_group_{i}"])
    predictions_test_groups.append(predictions)

# Evaluation using the metric
metric = evaluate.load("glue", "mrpc")
for i, predictions in enumerate(predictions_test_groups, start=1):
    preds = np.argmax(predictions.predictions, axis=-1)
    print(f"group {i}")
    print(metric.compute(predictions=preds, references=predictions.label_ids))


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6800 [00:00<?, ? examples/s]

Step,Training Loss
500,0.692400
1000,0.613300
1500,0.487500
2000,0.448400
2500,0.387700


Checkpoint destination directory test-trainer/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


group 1
{'accuracy': 0.8869565217391304, 'f1': 0.0}
group 2
{'accuracy': 0.7695652173913043, 'f1': 0.8697788697788698}
group 3
{'accuracy': 0.7608695652173914, 'f1': 0.0}
group 4
{'accuracy': 0.8956521739130435, 'f1': 0.944954128440367}


In [ ]:
print("group 2")
metric.compute(predictions=preds_test_group_2, references=predictions_test_group_2.label_ids)

group 2


{'accuracy': 0.0, 'f1': 0.0}

In [ ]:
print("group 3")
metric.compute(predictions=preds_test_group_3, references=predictions_test_group_3.label_ids)

group 3


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


{'accuracy': 1.0, 'f1': 0.0}

In [ ]:
print("group 4")
metric.compute(predictions=preds_test_group_4, references=predictions_test_group_4.label_ids)

group 4


{'accuracy': 0.0, 'f1': 0.0}